In [2]:
import datetime
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
sns.set_theme(context='talk', style='whitegrid', palette='deep')
plt.rcParams['figure.figsize'] = 10, 7
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 25
plt.rcParams['figure.titlesize'] = 32
plt.rcParams['axes.titlesize'] = 32
plt.rcParams['savefig.format'] = 'pdf'
plt.rcParams['figure.autolayout'] = 'true'
plt.rcParams['figure.frameon'] = 'false'
plt.rcParams['axes.spines.left'] = 'false'
plt.rcParams['axes.spines.right'] = 'false'
plt.rcParams['axes.spines.top'] = 'false'
plt.rcParams['legend.fancybox'] = 'false'
plt.rcParams['axes.spines.bottom'] = 'false'
plt.rcParams['font.size'] = 20
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
# для графиков, где надо много цветов, юзайте воть:
sns.set_palette(sns.color_palette('deep'))
# а по дефолту воть:
sns.set_palette(sns.color_palette('BuGn_r', n_colors=10)[2::3])
pd.set_option('display.max_columns', 60)

In [3]:
feeding_details_22 = pd.read_csv('datasets/2022-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_22 = pd.read_csv('datasets/2022-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_23 = pd.read_csv('datasets/2023-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_23 = pd.read_csv('datasets/2023-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_24 = pd.read_csv('datasets/2024-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_24 = pd.read_csv('datasets/2024-feeding-tasks.csv', on_bad_lines='skip')

feeding_details_25 = pd.read_csv('datasets/2025-feeding-tasks-details.csv', on_bad_lines='skip')
feeding_25 = pd.read_csv('datasets/2025-feeding-tasks.csv', on_bad_lines='skip')

In [4]:
monthly_feeding = pd.read_excel('smth.xlsx', sheet_name='Feeding (monthly)', header=1, skiprows=[2])
herd_metrics = pd.read_excel('smth.xlsx', sheet_name='Herd maintenance', header=1, skiprows=[2]).replace('-', np.nan)
production_indicators = pd.read_excel('smth.xlsx', sheet_name='Dairy indicators', header=1, skiprows=[2]).replace('-', np.nan)

In [5]:
feeding_details_23.IngredientName.unique()

array(['Солома (общ.)', 'Кукуруза', 'Комбикорм 10 группы', 'Шрот соевый',
       'Шрот рапсовый', 'Кукуруза плющеная', '2201.02.01.01.2.22',
       '2201.03.05.02.1.22', 'Патока свекловичная', 'Вода',
       'Комбикорм 2 группы', 'Сено луговое', 'Комбикорм 3.1 группы',
       'Премикс Молодняк 9-12', 'Остатки', 'Молоко', 'ЗЦМ',
       'Шаумацид Дринк', 'Электролит', 'Стартер', 'Бустер Милк',
       'Биотек Микс', '2221.01.01.01.1.22', '2221.03.05.02.1.22',
       '5902.01.01.01.1.22', 'Жом свекловичный сухой',
       '5313.06.05.02.1.22', '2221.04.05.02.1.21', '2501.03.01.01.1.22',
       'Комбикорм №11 ЭНА Восток', '2501.06.05.02.1.22',
       '3636.01.03.01.1.21', '2501.04.10.01.1.22',
       'Комбикорм № 3 престартер гранул.', '3645.01.01.01.1.22', 'Сорго',
       'Комбикорм 11 группы', '3645.03.05.02.1.22', 'Соль',
       'Премикс Дойный А', 'Сода', 'Пшеница', '5204.01.01.01.1.22',
       '5100.01.01.01.1.22', '5001.02.05.02.1.22', 'Мел',
       'Премикс Транзит А', 'Мегабуст Румен

In [6]:
def merge_tables() -> pd.DataFrame:
    """
    Merges all feeding dataframes with all details dataframes
    returns: merged dataframe
    rtype: pd.DataFrame
    """
    feeding_22["year"] = "2022"
    feeding_23["year"] = "2023"
    feeding_24["year"] = "2024"
    feeding_25["year"] = "2025"
    feeding_details_22["year"] = "2022"
    feeding_details_23["year"] = "2023"
    feeding_details_24["year"] = "2024"
    feeding_details_25["year"] = "2025"

    feeding_all = pd.concat([feeding_22, feeding_23, feeding_24, feeding_25], ignore_index=True)
    feeding_details_all = pd.concat([feeding_details_22, feeding_details_23, feeding_details_24, feeding_details_25], ignore_index=True)
    
    return feeding_all.merge(feeding_details_all, on=["FeedingTaskID", "SectionID", "year", "PhysiologicalGroupID", "PhysiologicalGroupName"], how="left"), feeding_all, feeding_details_all

In [7]:
monthly = pd.read_csv('datasets/monthly_vectors.csv')

In [8]:
feeding_and_details, feeding_all, feeding_details_all = merge_tables()

In [9]:
monthly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7245 entries, 0 to 7244
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              7245 non-null   int64  
 1   year_month              7245 non-null   object 
 2   FarmName                7245 non-null   object 
 3   PhysiologicalGroupName  7245 non-null   object 
 4   Concentrate             7245 non-null   float64
 5   Forage                  7245 non-null   float64
 6   VitaminMineral          7245 non-null   float64
 7   Water                   7245 non-null   float64
 8   pca1                    7245 non-null   float64
 9   pca2                    7245 non-null   float64
dtypes: float64(6), int64(1), object(3)
memory usage: 566.1+ KB


In [10]:
feeding_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2155378 entries, 0 to 2155377
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   FeedingTaskID                object 
 1   Date                         object 
 2   FarmName                     object 
 3   FeedNumber                   int64  
 4   SectionID                    int64  
 5   PhysiologicalGroupID         int64  
 6   PhysiologicalGroupName       object 
 7   PhysiologicalGroupHeadCount  int64  
 8   Appetite                     float64
 9   RationName                   object 
 10  RationPart                   float64
 11  TotalWeight_kg               float64
 12  CompletedAt                  object 
 13  year                         object 
dtypes: float64(3), int64(4), object(7)
memory usage: 230.2+ MB


In [11]:
feeding_details_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16443079 entries, 0 to 16443078
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   FeedingTaskID           object 
 1   SectionID               int64  
 2   PhysiologicalGroupID    int64  
 3   PhysiologicalGroupName  object 
 4   IngredientID            int64  
 5   IngredientName          object 
 6   IngredientType          object 
 7   PhysicalWeight_kg       float64
 8   year                    object 
dtypes: float64(1), int64(3), object(5)
memory usage: 1.1+ GB


In [12]:
feeding_details_all.IngredientName.unique().shape

(1852,)

In [13]:
import re
from collections import Counter
def normalize_name(s):
    if pd.isna(s):
        return ""
    s = str(s).strip()
    s = s.replace("//", "/")
    s = re.sub(r"\s+", " ", s)
    s = s.lower()
    return s
feeding_details_all['norm_ingr_name'] = feeding_details_all['IngredientName'].apply(normalize_name)

In [14]:
code_re = re.compile(r'^\d+(?:\.\d+)+$') #Честно сам писал (100%)
feeding_details_all['is_code'] = feeding_details_all['norm_ingr_name'].str.match(code_re)
feeding_details_all['is_code'].sum()

np.int64(5111456)

In [15]:
feeding_details_all['norm_ingr_name']

0                  солома покупная
1               3645.01.01.01.1.20
2              комбикорм 10 группы
3                   кукуруза сухая
4                шрот подсолнечный
                     ...          
16443074             солома (общ.)
16443075        6203.01.01.01.1.25
16443076    жом свекловичный сухой
16443077     премикс молодняк 6-24
16443078        6202.03.05.02.1.24
Name: norm_ingr_name, Length: 16443079, dtype: object

In [16]:
smth = pd.ExcelFile("smth.xlsx")
nsi = smth.sheet_names[0]

In [17]:
cultures = pd.read_excel(
    smth,
    sheet_name=nsi,
    usecols="B:C",
    skiprows=81,
    nrows=40 
)
cultures.columns = ["code", "name"]
cultures.head()

,code,name
0,1.0,Люцерна
1,2.0,Эспарцет
2,3.0,Многолетние травы
3,4.0,Луговые травы
4,5.0,Кукуруза


In [18]:
razdels = pd.read_excel(
    smth,
    sheet_name=nsi,
    usecols="E:H",
    skiprows=81,
    nrows=88  
)
razdels.columns = ["code", "region", "prop_farm_name", "farm_name"]
razdels.head()


,code,region,prop_farm_name,farm_name
0,2501,Воронеж,ЭНА Восток,ЖК Бобров
1,2371,Воронеж,ЭНА Восток,ЖК Верхний Икорец
2,2281,Воронеж,ЭНА Восток,МТФ Владимировка
3,2351,Воронеж,ЭНА Восток,МТФ Петропавловка
4,3637,Воронеж,ЭНА Восток,ЖК Бобров 2


In [19]:
feed_type = pd.read_excel(
    smth,
    sheet_name=nsi,
    usecols="B:C",
    skiprows=123,
    nrows=10
)
feed_type.columns = ["code", "feed_name"]
feed_type.head()

,code,feed_name
0,1.0,Сенаж
1,2.0,Силос
2,3.0,Сено
3,4.0,Солома
4,5.0,Концентраы


In [36]:
cultures_map = cultures.set_index('code')['name'].to_dict()
feed_type_map = feed_type.set_index('code')['feed_name'].to_dict()

In [ ]:
def decode_ingr_code(code):
    if pd.isna(code) or not isinstance(code, str):
        return code
    parts = code.split('.')
    if len(parts) < 4:
        return code
    try:
        culture_code = float(parts[2])
        feed_code = float(parts[3])
    except ValueError:
        return code    
    culture_name = cultures_map.get(culture_code, str(culture_code))
    feed_name = feed_type_map.get(feed_code, str(feed_code))
    return f"{culture_name} {feed_name}".lower()

In [34]:
feed_type_map

{1.0: 'Сенаж',
 2.0: 'Силос',
 3.0: 'Сено',
 4.0: 'Солома',
 5.0: 'Концентраы',
 6.0: 'Консервированное зерно',
 7.0: 'Корнаж',
 8.0: 'Сенаж в плёнке',
 9.0: 'Сенаж органический',
 nan: nan}

In [38]:
feeding_details_all['decoded_name'] = feeding_details_all.apply(
    lambda row: decode_ingr_code(row['norm_ingr_name']) if row['is_code'] else row['norm_ingr_name'],
    axis=1
)

feeding_details_all[['norm_ingr_name', 'decoded_name']].head(20)

,norm_ingr_name,decoded_name
0,солома покупная,солома покупная
1,3645.01.01.01.1.20,Люцерна Сенаж
2,комбикорм 10 группы,комбикорм 10 группы
3,кукуруза сухая,кукуруза сухая
4,шрот подсолнечный,шрот подсолнечный
5,шрот соевый,шрот соевый
6,жмых рапсовый,жмых рапсовый
7,жир защищённый,жир защищённый
8,2262.01.05.02.1.21,Кукуруза Силос
9,вода,вода


In [59]:
GROUP_KEYWORDS = {
    "forage": ["силос", "сенаж", "сено", "люцерн", "сенаж", "солома", "зеленая масса", "зел масса", "сило", "сенаж", 'сорго'],
    "energy": ["кукуруза", "пшениц", "ячмень", "овёс", "кукуруза плющ", "зерно", "зerno", "концентрат углевод", "глюгтен", "глютен", "корнаж", "карнаж", "карнаж", 'патока свекловичная', 'глицерин', 'патока', 'с-с кукурузный 1'],
    "protein": ["шрот", "жмых", "соев", "соевый", "протеин", "оболочка сои", "соя", "жмых", "люпин", "горох", "бобы", "белок", 'лен'],
    "fat": ["жир", "масло", "пропионат", "жир защищённый", "масло соевое"],
    "premix": ["премикс", "премикс дойн", "премикс молод", "премикс транзит", "премикс"],
    "mineral_vit": ["мел", "соль", "витамин", "минерал", "известняк", "кальций", "фосфат", "лизунец",  'минвит', 'сода', 'поташ', 'галит брикеты'],
    "additive": ["дрожж", "пробиот", "фермент", "фермент", "энзим", "энзимы", "буфер", "аквасейф", "биотек", "мегабуст", "электролит", 'кисол', 'шаумацид дринк'],
    "byproduct": ["жом", "барда", "жом свекловичный", "сыворотка", "дробина", "зeрноотход", "остатк", "остатки"],
    "water": ["вода","влажн","сыворотка молочная","молоко"],
    "premix_blend": ["комбикорм", "кк", "кк№", "кк №", "комбикорм №", "комбикорм", 'стартер', 'престартер', 'бустер милк', 'кормосмесь №10 снт', 'кальвобустер', 'бвмк для коров', 'зцм']
}

In [ ]:
def classify_ingredient(name):
    if not isinstance(name, str):
        return "other"
    name_lower = name.lower()
    for group, kat_list in GROUP_KEYWORDS.items():
        for kat in kat_list:
            if kat in name_lower:
                return group
    return "other"
feeding_details_all['ingredient_group'] = feeding_details_all['decoded_name'].apply(classify_ingredient)
feeding_details_all[['norm_ingr_name', 'decoded_name', 'ingredient_group']].head(20)

,norm_ingr_name,decoded_name,ingredient_group
0,солома покупная,солома покупная,forage
1,3645.01.01.01.1.20,Люцерна Сенаж,forage
2,комбикорм 10 группы,комбикорм 10 группы,premix_blend
3,кукуруза сухая,кукуруза сухая,energy
4,шрот подсолнечный,шрот подсолнечный,protein
5,шрот соевый,шрот соевый,protein
6,жмых рапсовый,жмых рапсовый,protein
7,жир защищённый,жир защищённый,fat
8,2262.01.05.02.1.21,Кукуруза Силос,forage
9,вода,вода,water


In [61]:
print(feeding_details_all.ingredient_group.value_counts())

ingredient_group
forage          5949918
energy          2862586
protein         2663553
premix_blend    1016786
water            971809
byproduct        962679
premix           859104
mineral_vit      670766
fat              264102
additive         190050
other             31726
Name: count, dtype: int64


In [62]:
others = feeding_details_all[feeding_details_all['ingredient_group'] == 'other']
others['decoded_name'].value_counts().head(10)

decoded_name
с-с кукурузный 01           2990
с-ж клевер-тимофеевка 05    2983
мука кукурузная             2795
c-ж клевер-тимофеевка 04    2212
с-ж рожь 4                  1988
зск                         1984
с-с кукурузный алтай        1138
02.01                       1060
с-с кукурузный огнево       1054
03.05                        947
Name: count, dtype: int64